# 사용자의 목표 Kcal계산식
- PA 선택: select_pa_by_goal로 체중 차이에 따라 PA를 동적으로 설정합니다.
- 기본 EER 계산: calculate_eer로 IOM 공식을 적용해 유지 칼로리를 산출합니다.

- 목표 계수: calculate_goal_factor로 체중 감량(또는 증량) 목표에 따라 5~20% 범위에서 칼로리를 줄이거나 늘립니다.

- 최종 EER: 유지 칼로리에 목표 계수를 곱해 개인화된 하루 칼로리 요구량을 도출합니다.

- 예시에서 weight=90kg, targetWeight=40kg인 경우 PA는 1.0, 목표 계수는 0.80(20% 감량), 결과 개인화 EER은 약 2261.5 kcal/day가 됩니다.

- 체중 증량이 목표면 diff < 0로 goal_factor > 1이 적용되어 더 높은 칼로리가 산출됩니다.

In [6]:
# Enhanced EER calculation reflecting distinct goals for weight loss vs. gain

def select_pa_by_goal(weight, target_weight, gender):
    """
    Select PA based on weight goal:
    - diff >= 20: sedentary
    - 10 <= diff < 20: low active
    - 0 < diff < 10: active
    - diff <= 0: very active
    """
    diff = weight - target_weight
    if diff >= 20:
        return 1.0  # sedentary
    elif diff >= 10:
        return 1.11 if not gender else 1.12  # low active
    elif diff > 0:
        return 1.25 if not gender else 1.27  # active
    else:
        return 1.48 if not gender else 1.45  # very active

def calculate_eer(age, gender, weight_kg, height_cm, pa):
    """
    Calculate base EER using IOM formula with numeric PA.
    """
    height_m = height_cm / 100.0
    if not gender:
        return 662 - (9.53 * age) + pa * ((15.91 * weight_kg) + (539.6 * height_m))
    else:
        return 354 - (6.91 * age) + pa * ((9.36 * weight_kg) + (726 * height_m))

def calculate_goal_factor(weight, target_weight):
    """
    Compute goal adjustment factor:
    - Weight loss: reduce intake by 5–20%
    - Weight gain: increase intake by 5–20%
    """
    diff = weight - target_weight
    base = abs(diff) * 0.005
    factor = min(max(base, 0.05), 0.20)  # bound 5% to 20%
    return 1 - factor if diff > 0 else 1 + factor

def calculate_personalized_eer(request_data):
    user = request_data['user']
    age = user['age']
    gender = user['gender']
    weight = user['weight']
    target_weight = user['targetWeight']
    height = user['height']
    
    # 1. PA based on goal
    pa = select_pa_by_goal(weight, target_weight, gender)
    
    # 2. Base EER
    base_eer = calculate_eer(age, gender, weight, height, pa)
    
    # 3. Goal factor
    goal_factor = calculate_goal_factor(weight, target_weight)
    
    # 4. Final personalized EER
    personalized_eer = base_eer * goal_factor
    
    return {
        'personalized_eer': personalized_eer,
        'pa_used': pa,
        'goal_factor': goal_factor
    }

In [8]:


# Example
request_data = {
    'user': {
        'age': 25,
        'gender': False,
        'weight': 90,
        'targetWeight': 60,
        'height': 180
    }
}

result = calculate_personalized_eer(request_data)
print(f"PA used: {result['pa_used']}")
print(f"Goal factor: {result['goal_factor']:.2f}")
print(f"Personalized EER: {result['personalized_eer']:.2f} kcal/day")


PA used: 1.0
Goal factor: 0.85
Personalized EER: 2402.89 kcal/day


# AMDR 기반 3대 영양소 권장 비율
1.1 탄수화물(Carbohydrate)
- 권장 비율: 총 에너지의 45–65% 
- 근거: IOM에서는 저열량 식이 시 높은 지방 섭취를 방지하고 필수 에너지원 확보를 위해 최소 45%, 만성질환 위험 증가를 억제하기 위해 최대 65%를 제시합니다 


1.2 단백질(Protein)
- 권장 비율: 총 에너지의 10–35% 
- 근거: AMDR은 근육 유지 및 생리 기능 보전을 위해 최소 10%, 과잉 섭취 시 신장 부담을 고려해 35%를 상한으로 설정했습니다 


1.3 지방(Fat)
- 권장 비율: 총 에너지의 20–35% 
- 추가 지침: 포화지방은 총 에너지의 ≤10%로 제한하고, 트랜스지방은 가능한 한 최소화(≤1%)하도록 권장합니다 


2. 자유당(Free Sugars) 권장 기준
- WHO 권고: 자유당 섭취를 총 에너지의 **<10%**로 제한하고, 추가 건강 이득을 위해 <5% 이하로 줄일 것을 권장합니다 
- 정의: 자유당은 가공식품 첨가당뿐만 아니라 꿀·시럽·과즙 농축액에 포함된 당도 포함합니다

In [9]:
# Enhanced EER calculation reflecting distinct goals for weight loss vs. gain

def select_pa_by_goal(weight, target_weight, gender):
    """
    Select PA based on weight goal:
    - diff >= 20: sedentary
    - 10 <= diff < 20: low active
    - 0 < diff < 10: active
    - diff <= 0: very active
    """
    diff = weight - target_weight
    if diff >= 20:
        return 1.0  # sedentary
    elif diff >= 10:
        return 1.11 if not gender else 1.12  # low active
    elif diff > 0:
        return 1.25 if not gender else 1.27  # active
    else:
        return 1.48 if not gender else 1.45  # very active

def calculate_eer(age, gender, weight_kg, height_cm, pa):
    """
    Calculate base EER using IOM formula with numeric PA.
    """
    height_m = height_cm / 100.0
    if not gender:
        return 662 - (9.53 * age) + pa * ((15.91 * weight_kg) + (539.6 * height_m))
    else:
        return 354 - (6.91 * age) + pa * ((9.36 * weight_kg) + (726 * height_m))

def calculate_goal_factor(weight, target_weight):
    """
    Compute goal adjustment factor:
    - Weight loss: reduce intake by 5–20%
    - Weight gain: increase intake by 5–20%
    """
    diff = weight - target_weight
    base = abs(diff) * 0.005
    factor = min(max(base, 0.05), 0.20)  # bound 5% to 20%
    return 1 - factor if diff > 0 else 1 + factor

def calculate_personalized_eer(request_data):
    user = request_data['user']
    age = user['age']
    gender = user['gender']
    weight = user['weight']
    target_weight = user['targetWeight']
    height = user['height']
    
    # 1. PA based on goal
    pa = select_pa_by_goal(weight, target_weight, gender)
    
    # 2. Base EER
    base_eer = calculate_eer(age, gender, weight, height, pa)
    
    # 3. Goal factor
    goal_factor = calculate_goal_factor(weight, target_weight)
    
    # 4. Final personalized EER
    personalized_eer = base_eer * goal_factor
    
    return {
        'personalized_eer': personalized_eer,
        'pa_used': pa,
        'goal_factor': goal_factor
    }

# — 여기에 목표 칼로리 기반 매크로 계산 함수 추가 — 

def calculate_macro_targets(personalized_eer):
    """
    Calculate recommended gram ranges for macros based on AMDR and WHO sugar guidelines.
    Returns a dict:
      {
        'carbohydrate': {'min_g':…, 'max_g':…},
        'protein':      {'min_g':…, 'max_g':…},
        'fat':          {'min_g':…, 'max_g':…},
        'sugar':        {'max_g':…}   # 최소는 0으로 고정
      }
    """
    # AMDR 비율 범위 및 자유당 상한(%)
    ratios = {
        'carbohydrate': (0.45, 0.65),  # 총 에너지 대비 45–65% :contentReference[oaicite:4]{index=4}
        'protein':      (0.10, 0.35),  # 총 에너지 대비 10–35% :contentReference[oaicite:5]{index=5}
        'fat':          (0.20, 0.35),  # 총 에너지 대비 20–35% :contentReference[oaicite:6]{index=6}
    }
    sugar_max_ratio = 0.10          # 자유당 <10% :contentReference[oaicite:7]{index=7}
    
    # 1g당 칼로리
    cal_per_g = {'carbohydrate': 4, 'protein': 4, 'fat': 9, 'sugar': 4}
    
    targets = {}
    for macro, (low_ratio, high_ratio) in ratios.items():
        low_kcal  = personalized_eer * low_ratio
        high_kcal = personalized_eer * high_ratio
        targets[macro] = {
            'min_g': low_kcal  / cal_per_g[macro],
            'max_g': high_kcal / cal_per_g[macro]
        }
    # 당류: 최소 0g, 최대 personalized_eer * sugar_max_ratio / 4g
    sugar_max_kcal = personalized_eer * sugar_max_ratio
    targets['sugar'] = {
        'min_g': 0,
        'max_g': sugar_max_kcal / cal_per_g['sugar']
    }
    return targets

# Example 전체 흐름
request_data = {
    'user': {
        'age': 25,
        'gender': False,       # 남성
        'weight': 90,
        'targetWeight': 40,
        'height': 180
    }
}


In [10]:
# 1) 개인화 EER 계산
eer_result = calculate_personalized_eer(request_data)
eer_value = eer_result['personalized_eer']

# 2) 매크로 목표량 계산
macro_targets = calculate_macro_targets(eer_value)

print(f"Personalized EER: {eer_value:.2f} kcal/day")
print("Macro targets (g/day):")
for m, vals in macro_targets.items():
    if m == 'sugar':
        print(f"  {m}: max {vals['max_g']:.1f}g")
    else:
        print(f"  {m}: {vals['min_g']:.1f}–{vals['max_g']:.1f} g")


Personalized EER: 2261.54 kcal/day
Macro targets (g/day):
  carbohydrate: 254.4–367.5 g
  protein: 56.5–197.9 g
  fat: 50.3–87.9 g
  sugar: max 56.5g


In [12]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Constants for category combos
CATEGORY_COMBOS = [
    ["국 및 탕류", "생채·무침류"], ["국 및 탕류", "볶음류"], 
    # ... (생략, 이전에 정의한 나머지 콤보 포함)
]
SINGLE_CATEGORIES = ["면 및 만두류", "죽 및 스프류"]

# 1. Load data
# Assuming 'foods_info.csv' has columns: id, name, category, serving_size_g, energy_kcal
# and 'foods_nutrients.csv' has columns: id, carbohydrate_g, protein_g, fat_g, sugar_g (per 100g)
df_info = pd.read_csv('../data/foods_info.csv')
df_nutri = pd.read_csv('../data/foods_nutrients.csv')
df = pd.merge(df_info, df_nutri, on='id')

# 2. Compute actual nutrients per serving
nutrients = ['carbohydrate_g', 'protein_g', 'fat_g', 'sugar_g']
for nut in nutrients:
    df[f'{nut}_serving'] = df[nut] * df['serving_size_g'] / 100

# 3. Retrieve targets (from earlier calculation)
# personalized_eer and macro_targets should be computed before this step
# For example:
# eer_value = 2261.54
# macro_targets = {
#   'carbohydrate': {'min_g': 250, 'max_g': 350},
#   'protein':      {'min_g': 60,  'max_g': 200},
#   'fat':          {'min_g': 50,  'max_g': 80},
#   'sugar':        {'min_g': 0,   'max_g': 56}
# }
eer_value = eer_value  # from calculate_personalized_eer
macro_targets = macro_targets  # from calculate_macro_targets

# 4. Filter by energy constraint (e.g., remaining calories for dinner)
# Assume lunch_calories is known; remain_cal = eer_value - lunch_calories
remain_cal = eer_value - request_data['nutrients']['calories']
df_filtered = df[df['energy_kcal'] <= remain_cal]

# 5. Build feature & target vectors for content-based scoring
target_vector = np.array([
    (macro_targets['carbohydrate']['min_g'] + macro_targets['carbohydrate']['max_g']) / 2,
    (macro_targets['protein']['min_g'] + macro_targets['protein']['max_g']) / 2,
    (macro_targets['fat']['min_g'] + macro_targets['fat']['max_g']) / 2,
    macro_targets['sugar']['max_g']  # use max sugar as limit
]).reshape(1, -1)

feature_matrix = df_filtered[
    [f'{nut}_serving' for nut in nutrients]
].values

# 6. Compute cosine similarity scores
scores = cosine_similarity(feature_matrix, target_vector).flatten()
df_filtered['score'] = scores

# 7. Apply category combination rules and select top recommendations
recommendations = []
for combo in CATEGORY_COMBOS:
    cat1, cat2 = combo
    d1 = df_filtered[df_filtered['category'] == cat1]
    d2 = df_filtered[df_filtered['category'] == cat2]
    if not d1.empty and not d2.empty:
        top1 = d1.sort_values('score', ascending=False).iloc[0]
        top2 = d2.sort_values('score', ascending=False).iloc[0]
        recommendations.append({
            'combo': combo,
            'foods': [top1['name'], top2['name']],
            'scores': [top1['score'], top2['score']]
        })

# 8. Handle single categories if no combo found
for cat in SINGLE_CATEGORIES:
    d = df_filtered[df_filtered['category'] == cat]
    if not d.empty:
        top = d.sort_values('score', ascending=False).iloc[0]
        recommendations.append({
            'combo': [cat],
            'foods': [top['name']],
            'scores': [top['score']]
        })

# Display recommendations
for rec in recommendations:
    print(f"Combo: {rec['combo']}")
    for food, sc in zip(rec['foods'], rec['scores']):
        print(f"  - {food}: score {sc:.3f}")



KeyError: 'id'